In [ ]:
####################### Random Forest scripts adapted from:
#### https://towardsdatascience.com/machine-learning-step-by-step-6fbde95c455a

In [67]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
norm_e = StandardScaler()

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'sklearn'

In [69]:
### set work directory
os.chdir("/pscratch/sd/j/jdyuzon/snakemake-bacteriophage2")

In [105]:
genes_phage_tail_variables=pd.read_csv("prediction_out/genes_phage_tail_variables.csv")
y=genes_phage_tail_variables['type']
y=y.replace('Phage1',1)
y=y.replace('Phage2',1)
y=y.replace('T6SS',0)
y=y.replace('eCIS',0)
X=genes_phage_tail_variables.drop(['viral_ID','type'], axis =1, inplace=False)
genes_phage_tail_variables=genes_phage_tail_variables.drop(columns=['Unnamed: 0'])
X=X.drop(columns=['Unnamed: 0'])
X

,COG0009,COG0025,COG0027,COG0030,COG0031,COG0034,COG0037,COG0103,COG0116,COG0118,...,TIGR04517,TIGR04528,TIGR04548,TIGR04551,TIGR04558,TIGR04559,TIGR04563,TIGR04565,TIGR04569,TIGR04571
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
### Perform the train-test split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.25, stratify=y)
print(X_train.shape)
### No need to transform binary variables
### Write tables
X_train.to_csv('prediction_out/random_forest/X_train.csv')
X_test.to_csv('prediction_out/random_forest/X_test.csv')
y_train.to_csv('prediction_out/random_forest/y_train.csv')
y_test.to_csv('prediction_out/random_forest/y_test.csv')

### Are there overlaps in test and train datasets? train_test_split should sample without replacement so no overlaps
pd.merge(X_train, X_test, how='inner')

(1554, 4752)


,COG0009,COG0025,COG0027,COG0030,COG0031,COG0034,COG0037,COG0103,COG0116,COG0118,...,TIGR04517,TIGR04528,TIGR04548,TIGR04551,TIGR04558,TIGR04559,TIGR04563,TIGR04565,TIGR04569,TIGR04571
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
### Load data 
#X_train = pd.read_csv('prediction_out/random_forest/X_train.csv')
#X_test = pd.read_csv('prediction_out/random_forest/X_test.csv')
#y_train = pd.read_csv('prediction_out/random_forest/y_train.csv')
#y_test = pd.read_csv('prediction_out/random_forest/y_test.csv')

In [107]:
X_train

,COG0009,COG0025,COG0027,COG0030,COG0031,COG0034,COG0037,COG0103,COG0116,COG0118,...,TIGR04517,TIGR04528,TIGR04548,TIGR04551,TIGR04558,TIGR04559,TIGR04563,TIGR04565,TIGR04569,TIGR04571
447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
y_train

447     1
1037    1
320     1
1371    1
1843    0
       ..
1589    1
2064    0
1845    0
1501    1
361     1
Name: type, Length: 1554, dtype: int64

In [109]:
###instantiate the model and fit the scaled data to it
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train, y_train)
# Test score (accuracy) for training data
rfc.score(X_train, y_train)

1.0

In [110]:
# Test score (accuracy) for test data
rfc.score(X_test, y_test)

0.9922928709055877

In [111]:
# View the classification report for test data and predictions
y_pred_test = rfc.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        72
           1       0.99      1.00      1.00       447

    accuracy                           0.99       519
   macro avg       1.00      0.97      0.98       519
weighted avg       0.99      0.99      0.99       519



In [112]:
importances = rfc.feature_importances_
features=X_train

In [113]:
### Feature importance based on GINI/Mean Decrease in Impurity (MDI)
feats = {}
for feature, importance in zip(X_train, rfc.feature_importances_):
    feats[feature] = importance
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
importances = importances.sort_values(by='Gini-Importance', ascending=False)
importances = importances.reset_index()
importances = importances.rename(columns={'index': 'Features'})
importances[0:10]

,Features,Gini-Importance
0,PF18945,0.025707
1,TIGR03353,0.024965
2,PF17482,0.024407
3,COG3519,0.022296
4,TIGR03359,0.022212
5,PF19265,0.020410
6,COG3522,0.020053
7,TIGR03347,0.018458
8,K11893,0.018173
9,COG3520,0.017491


In [114]:
importances.to_csv('prediction_out/random_forest/GINI_importances.csv', sep='\t')